In [2]:
import chess.pgn
import sys

f1 = open('white_wins.txt', 'w')
f2 = open('black_wins.txt', 'w')
pgn = open("train.pgn")
n_games = 0

while True:
    game = chess.pgn.read_game(pgn)
    try:
        result = game.headers["Result"]
    except:
        break

    if result == "1-0":
        n_games =+ 1
        board = game.board()
        for move in game.main_line():
            board.push(move)
            f1.write(str(board))
            f1.write("\n- - - - - - - -\n")
        f1.write("\n= = = = = = = =\n")
    if result == "0-1":
        n_games =+ 1
        board = game.board()
        for move in game.main_line():
            board.push(move)
            f2.write(str(board))
            f2.write("\n- - - - - - - -\n")
        f2.write("\n= = = = = = = =\n")

f1.close()        
f2.close()
pgn.close()
print("Done. ", n_games, " games parsed")

ModuleNotFoundError: No module named 'chess'

In [3]:
import tensorflow as tf
white = open('white_wins.txt', 'r')
black = open('black_wins.txt', 'r')



In [4]:
import numpy as np

# convert all data to a list form 
i = 0
boards = []
for line in white:
    temp = line.strip().split(" ")
    boards.append(temp)
# tst = np.load("A00-139_first_10000.npz")
# boards = tst['arr_0']
# targets = tst['arr_1']
# print(boards.shape)
# print(boards[0,1,:,:])
# print(targets.shape)
# print(boards)
boards_data = np.asarray(boards)

black_boards = []
for line in black:
    temp = line.strip().split(" ")
    black_boards.append(temp)


In [5]:
# First lets create a 8x8 boards
i = 0
boards_final = []
checklist = ['r','n', 'b', 'q', 'k', 'p', 'R','N', 'B', 'Q', 'K', 'P', '.']

for i in range(len(boards_data)):
    if any(check in boards_data[i] for check in checklist):
        boards_final.append(boards_data[i])
    
# print(len(boards_final))
# print(boards_final)
bbf = []
bb = np.asarray(black_boards)
for i in range(len(bb)):
    if any(check in bb[i] for check in checklist):
        bbf.append(bb[i])

In [6]:
# restructure the white boards to a chess board shape
boards_np = np.asarray(boards_final)
boards_np = boards_np.reshape((int(boards_np.shape[0]/8),8,8))

# restructure the black boards data to a chess board shape

bbf_np = np.asarray(bbf)
bbf_np = bbf_np.reshape((int(bbf_np.shape[0]/8),8,8))

# print(boards_np.shape)

In [9]:
# print(boards_np[0])
# convert to a map of 6x8x8 data type with the extra channel for 
# mapping the 6 types of chess pieces
boards_res = []

for i in range(len(boards_np)):
    test = boards_np[i]
    temp1 = np.zeros(test.shape)
    temp2 = np.zeros(test.shape)
    temp3 = np.zeros(test.shape)
    temp4 = np.zeros(test.shape)
    temp5 = np.zeros(test.shape)
    temp6 = np.zeros(test.shape)
    temp = []
    for i in range(len(test)):
        for j in range(len(test[0])):
            # for the rooks
            if test[i][j] == 'r':
                temp1[i][j] = -1
            if test[i][j] == 'R':
                temp1[i][j] = 1

            #for the n's
            if test[i][j] == 'n':
                temp2[i,j] = -1
            if test[i,j] == 'N':
                temp2[i,j] = 1

             #for the bishop
            if test[i][j] == 'b':
                temp3[i,j] = -1
            if test[i,j] == 'B':
                temp3[i,j] = 1

             #for the queen
            if test[i,j] == 'q':
                temp4[i,j] = -1
            if test[i,j] == 'Q':
                temp4[i,j] = 1

             #for the king
            if test[i][j] == 'k':
                temp5[i,j] = -1
            if test[i,j] == 'K':
                temp5[i,j] = 1

             #for the pawns
            if test[i][j] == 'p':
                temp6[i,j] = -1
            if test[i,j] == 'P':
                temp6[i,j] = 1

    temp.append(temp1)
    temp.append(temp2)
    temp.append(temp3)
    temp.append(temp4)
    temp.append(temp5)
    temp.append(temp6)
    temp = np.asarray(temp)
    boards_res.append(temp)

    
boards_res = np.asarray(boards_res)
print(boards_res.shape)
# print(boards_res)
# print(temp1)
# print(temp2)
# print(temp3)
# print(temp4)
# print(temp5)
# print(temp6)

(216982, 6, 8, 8)


In [8]:
black_res = []

for i in range(len(bbf_np)):
    test = bbf_np[i]
    temp1 = np.zeros(test.shape)
    temp2 = np.zeros(test.shape)
    temp3 = np.zeros(test.shape)
    temp4 = np.zeros(test.shape)
    temp5 = np.zeros(test.shape)
    temp6 = np.zeros(test.shape)
    temp = []
    for i in range(len(test)):
        for j in range(len(test[0])):
            # for the rooks
            if test[i][j] == 'r':
                temp1[i][j] = -1
            if test[i][j] == 'R':
                temp1[i][j] = 1

            #for the n's
            if test[i][j] == 'n':
                temp2[i,j] = -1
            if test[i,j] == 'N':
                temp2[i,j] = 1

             #for the bishop
            if test[i][j] == 'b':
                temp3[i,j] = -1
            if test[i,j] == 'B':
                temp3[i,j] = 1

             #for the queen
            if test[i,j] == 'q':
                temp4[i,j] = -1
            if test[i,j] == 'Q':
                temp4[i,j] = 1

             #for the king
            if test[i][j] == 'k':
                temp5[i,j] = -1
            if test[i,j] == 'K':
                temp5[i,j] = 1

             #for the pawns
            if test[i][j] == 'p':
                temp6[i,j] = -1
            if test[i,j] == 'P':
                temp6[i,j] = 1

    temp.append(temp1)
    temp.append(temp2)
    temp.append(temp3)
    temp.append(temp4)
    temp.append(temp5)
    temp.append(temp6)
    temp = np.asarray(temp)
    black_res.append(temp)

    
black_res = np.asarray(black_res)
print(black_res.shape)
# print(black_res)

(201837, 6, 8, 8)


In [85]:
# boards_res is of the form (216982,6,8,8) which is the correct shape for white wins

white_input_data = boards_res # board position where white wins
black_input_data = black_res # board position where black wins
white_input_label = np.ones(boards_res.shape[0])
black_input_label = (np.zeros(black_res.shape[0]))
print(white_input_label.shape)
print(black_input_label.shape)
input_label = np.int32(np.concatenate((white_input_label[:512], black_input_label[:512])))
print(input_label.shape)
print(input_label)
input_data = np.float32(np.concatenate((white_input_data[:512], black_input_data[:512])))
print(input_data.shape)
print(type(input_data[0][0][0][0]))

eval_data = np.float32(np.concatenate((white_input_data[512:1024], black_input_data[512:1024])))
eval_labels = np.int32(np.concatenate((white_input_label[512:1024], black_input_label[512:1024])))

(216982,)
(201837,)
(1024,)
[1 1 1 ... 0 0 0]
(1024, 6, 8, 8)
<class 'numpy.float32'>


In [88]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_fn(features, labels, mode):
    # Convolution layer #1
    input_layer = tf.reshape(features, [-1,8,8,6])
    conv1 = tf.layers.conv2d(inputs=input_layer, 
                             filters=400,
                             kernel_size=[4,4],
                             padding="same",
                             activation=tf.nn.relu)
#     print(conv1.shape)
    # Pooling layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
#     print(pool1.shape)
    # print(type(pool1))

    # Convolution layer #2
    conv2 = tf.layers.conv2d(inputs=pool1,
                            filters=200,
                            kernel_size=[2,2],
                            padding="same",
                            activation=tf.nn.relu)
#     print(conv2.shape)
    # print(type(conv2))
    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)

#     print(pool2.shape)
    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 2*2*200])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

#     print("Dense:", dense.shape)
    # Logit layer
    logits = tf.layers.dense(inputs=dropout, units=2)
    
#     print("Logits:", logits.shape)
    # print(logits)

    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

def main(unused_argv):
    # restructure input data 
    labels = input_label
    # setting the mode to training right now
    # mode = tf.estimator.ModeKeys.TRAIN

    cnn_classifier = tf.estimator.Estimator(model_fn=cnn_fn, model_dir="cnn_model")

    tensor_to_log = {"probabilites":"softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensor_to_log, every_n_iter=50)

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x=input_data,
        y=labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)

    cnn_classifier.train(input_fn=train_input_fn,
                          steps=1000,
                          hooks=[logging_hook])

    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x=eval_data,
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = cnn_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

if __name__ == "__main__":
    tf.app.run()
# cnn(input_layer, labels, mode)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000197A64C6B70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn_model\model.ckpt-2
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3 into cnn_model\model.ckpt.


INFO:tensorflow:loss = 0.69452965, step = 103 (24.619 sec)
INFO:tensorflow:probabilites = [[0.51095116 0.48904887]
 [0.5009623  0.4990377 ]
 [0.48033005 0.51967   ]
 [0.5135951  0.48640493]
 [0.5002739  0.49972612]
 [0.51778305 0.48221698]
 [0.49349517 0.50650483]
 [0.48823583 0.51176417]
 [0.49106884 0.5089312 ]
 [0.5066057  0.49339435]
 [0.46221823 0.5377818 ]
 [0.484654   0.515346  ]
 [0.50169027 0.49830976]
 [0.50008184 0.49991813]
 [0.4909972  0.5090028 ]
 [0.5183155  0.48168445]
 [0.49668765 0.5033124 ]
 [0.50531095 0.49468905]
 [0.47788075 0.5221192 ]
 [0.47499838 0.5250016 ]
 [0.50750726 0.49249268]
 [0.49855962 0.5014403 ]
 [0.4903989  0.50960106]
 [0.51829237 0.4817076 ]
 [0.48582256 0.5141774 ]
 [0.49449444 0.5055056 ]
 [0.47360927 0.5263908 ]
 [0.4809642  0.51903576]
 [0.46552306 0.534477  ]
 [0.5033347  0.49666527]
 [0.5125797  0.48742032]
 [0.5113344  0.48866558]
 [0.4896222  0.51037776]
 [0.50934744 0.49065256]
 [0.49552625 0.5044738 ]
 [0.48124087 0.51875913]
 [0.487396

INFO:tensorflow:loss = 0.68715996, step = 303 (24.933 sec)
INFO:tensorflow:probabilites = [[0.49087143 0.5091286 ]
 [0.5191863  0.48081368]
 [0.4962716  0.50372833]
 [0.4660068  0.5339932 ]
 [0.50437313 0.4956268 ]
 [0.48487344 0.5151266 ]
 [0.48160186 0.5183981 ]
 [0.5003483  0.49965173]
 [0.4901782  0.50982183]
 [0.5194978  0.48050216]
 [0.4745017  0.52549833]
 [0.53259665 0.46740332]
 [0.46988267 0.53011733]
 [0.517449   0.48255104]
 [0.4860292  0.5139708 ]
 [0.491844   0.50815594]
 [0.47001588 0.52998406]
 [0.48678666 0.5132134 ]
 [0.5166631  0.48333687]
 [0.4851704  0.51482964]
 [0.50667304 0.49332687]
 [0.4970827  0.5029173 ]
 [0.5021582  0.49784186]
 [0.49328476 0.5067153 ]
 [0.49972257 0.50027746]
 [0.5275579  0.472442  ]
 [0.47962412 0.5203759 ]
 [0.48833662 0.5116633 ]
 [0.51875687 0.4812431 ]
 [0.49051726 0.50948274]
 [0.48458937 0.5154107 ]
 [0.4790768  0.5209232 ]
 [0.5052716  0.49472836]
 [0.50839126 0.4916088 ]
 [0.4813048  0.5186952 ]
 [0.5213351  0.47866488]
 [0.488748

INFO:tensorflow:loss = 0.67484814, step = 503 (26.390 sec)
INFO:tensorflow:probabilites = [[0.53518397 0.464816  ]
 [0.5202586  0.47974136]
 [0.52377677 0.47622326]
 [0.49888724 0.50111276]
 [0.49129355 0.5087064 ]
 [0.5004951  0.49950486]
 [0.49887684 0.5011232 ]
 [0.51334584 0.4866542 ]
 [0.50908935 0.4909107 ]
 [0.49601346 0.50398654]
 [0.51790804 0.482092  ]
 [0.47506237 0.5249377 ]
 [0.496367   0.503633  ]
 [0.47748804 0.5225119 ]
 [0.4969806  0.50301945]
 [0.4811519  0.5188481 ]
 [0.49466774 0.50533223]
 [0.4901554  0.5098446 ]
 [0.4906378  0.5093622 ]
 [0.5344566  0.46554333]
 [0.5043422  0.49565783]
 [0.5068625  0.49313754]
 [0.47179642 0.52820367]
 [0.50260967 0.4973903 ]
 [0.53296995 0.46703002]
 [0.4888197  0.5111804 ]
 [0.47942716 0.5205728 ]
 [0.4757775  0.5242225 ]
 [0.5088249  0.4911751 ]
 [0.5129281  0.48707193]
 [0.46076694 0.539233  ]
 [0.48198846 0.51801157]
 [0.5259451  0.47405484]
 [0.5354749  0.46452513]
 [0.50992864 0.49007133]
 [0.4536852  0.5463149 ]
 [0.506825

INFO:tensorflow:loss = 0.6722279, step = 703 (25.311 sec)
INFO:tensorflow:probabilites = [[0.51416886 0.4858311 ]
 [0.46523872 0.5347613 ]
 [0.5113712  0.48862883]
 [0.48247305 0.517527  ]
 [0.5055224  0.4944776 ]
 [0.52065176 0.4793482 ]
 [0.503469   0.49653104]
 [0.476394   0.52360606]
 [0.4962393  0.50376076]
 [0.46229488 0.5377051 ]
 [0.52082443 0.4791756 ]
 [0.46629578 0.5337042 ]
 [0.46379378 0.53620625]
 [0.4837018  0.5162982 ]
 [0.48176682 0.5182332 ]
 [0.4985206  0.5014794 ]
 [0.49259833 0.5074016 ]
 [0.5098465  0.4901535 ]
 [0.49866265 0.5013374 ]
 [0.4840141  0.5159859 ]
 [0.47873336 0.5212667 ]
 [0.5144432  0.48555672]
 [0.5058593  0.49414065]
 [0.5089294  0.49107063]
 [0.4734986  0.5265014 ]
 [0.48721734 0.5127827 ]
 [0.51557213 0.4844278 ]
 [0.47890082 0.52109915]
 [0.4877147  0.5122853 ]
 [0.5079541  0.4920459 ]
 [0.5157149  0.4842851 ]
 [0.49318674 0.5068133 ]
 [0.49564058 0.5043594 ]
 [0.50166386 0.49833614]
 [0.49934953 0.5006504 ]
 [0.47935098 0.520649  ]
 [0.5170281

INFO:tensorflow:loss = 0.6693889, step = 903 (27.693 sec)
INFO:tensorflow:probabilites = [[0.49933213 0.5006678 ]
 [0.52611333 0.4738866 ]
 [0.5216735  0.4783265 ]
 [0.50023776 0.49976218]
 [0.53343093 0.46656913]
 [0.53771734 0.4622826 ]
 [0.47154808 0.5284519 ]
 [0.4484627  0.55153733]
 [0.52639323 0.47360682]
 [0.50528187 0.49471813]
 [0.52545494 0.47454506]
 [0.4874371  0.51256293]
 [0.524375   0.47562495]
 [0.48360324 0.5163968 ]
 [0.49194893 0.50805116]
 [0.52145    0.47855002]
 [0.5036029  0.49639708]
 [0.4805559  0.5194441 ]
 [0.54367465 0.45632544]
 [0.47654212 0.52345794]
 [0.5032092  0.49679077]
 [0.5280948  0.47190523]
 [0.521444   0.47855592]
 [0.53027606 0.46972394]
 [0.5264593  0.47354075]
 [0.51332545 0.48667455]
 [0.4751281  0.52487195]
 [0.55475193 0.44524807]
 [0.5147901  0.48520994]
 [0.5165299  0.48347008]
 [0.50665355 0.49334645]
 [0.5340677  0.46593225]
 [0.50087065 0.49912935]
 [0.49366036 0.5063396 ]
 [0.46344927 0.53655064]
 [0.47402835 0.5259717 ]
 [0.5242535

SystemExit: 

C:\Users\Shashank\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
